In [1]:
import zipfile  # 处理压缩文件
import os

import pandas as pd  # 处理 csv 文件
import numpy as np
from matplotlib import pyplot as plt

# ----- 自定义模块
from utils.zipimage import AnnZ, ImageZ, Loader, Dataset
%matplotlib inline

In [ ]:
root = 

def unzip(root):
    # 解压 all.zip 数据集，并删除 all.zip
    source_path = os.path.join(root, 'all.zip')

    with zipfile.ZipFile(source_path) as fp:
        fp.extractall(root)
    os.remove(source_path)
        
unzip(root)

In [ ]:
root = 'E:\Data\Kaggle\dog_cat'
"""

"""

os.listdir(root)

In [ ]:
submit = pd.read_csv(os.path.join(root, 'sampleSubmission.csv'))
submit.head()

In [ ]:
class TrainSet(ImageZ):
    def __init__(self, root, dataType):
        super().__init__(root, dataType)
        self._get_class_file()
    def _get_class_file(self):
        class_names = set([os.path.split(name)[-1].split('.')[0] for name in trainZ.names])
        self.class_file = {name:{} for name in class_names}
        for name in trainZ.names:
            class_name, idx, _ = os.path.split(name)[-1].split('.')
            self.class_file[class_name][int(idx)] = self.class_file[class_name].get(
                int(idx), name)
        self.class_names = tuple(class_names)

In [ ]:
trainZ = TrainSet(root, 'train')

In [ ]:
cat = []
for idx in sorted(trainZ.class_file['cat']):
    cat.append((trainZ.class_file['cat'][idx], 'cat'))

In [ ]:
trainZ.class_file['cat'][11]

In [ ]:
from keras.preprocessing import image as kimage
from keras.applications.inception_v3 import preprocess_input
from keras.applications import InceptionV3
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D

In [ ]:
datagen = kimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False)

In [ ]:
def add_new_last_layer(base_model, nb_classes):
    """Add last layer to the convnet
    Args:
      base_model: keras model excluding top
      nb_classes: # of classes
    Returns:
      new keras model with last layer
    """
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(FC_SIZE, activation='relu')(x)
    predictions = Dense(nb_classes, activation='softmax')(x)
    model = Model(input=base_model.input, output=predictions)
    return model


# 迁移学习
def setup_to_transfer_learn(model, base_model):
    """Freeze all layers and compile the model"""
    for layer in base_model.layers:
        layer.trainable = False
    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    
# 微调
def setup_to_finetune(model):
    """Freeze the bottom NB_IV3_LAYERS and retrain the remaining top 
       layers.
    note: NB_IV3_LAYERS corresponds to the top 2 inception blocks in 
          the inceptionv3 architecture
    Args:
      model: keras model
    """
    for layer in model.layers[:NB_IV3_LAYERS_TO_FREEZE]:
        layer.trainable = False
    for layer in model.layers[NB_IV3_LAYERS_TO_FREEZE:]:
        layer.trainable = True
    model.compile(optimizer=SGD(lr=0.0001, momentum=0.9),
                  loss='categorical_crossentropy')